In [ ]:
# please check https://www.kaggle.com/ikobzev/moa-keras-nn-baseline-from-ml-newbie #updated on 14th Nov
# deal G separately and C separately for auto encoders

#https://www.kaggle.com/optimo/selfsupervisedtabnet --> please refer this 20/11

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import QuantileTransformer
!pip install ../input/pytorchtabnetpretraining/pytorch_tabnet-2.0.1-py3-none-any.whl
!pip install ../input/iterative-stratification/iterative-stratification-master/

#---------------- torch imports -----------------------------
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
#---------------------------------------------------------------
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# ---------------------------------------START OF RAPIDS installation---------------------
import sys
# !cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
# !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
# !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
# -----------------------------------------END of RAPIDS installation-------------------------
# !pip install iterative-stratification
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
#from cuml.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import os
import seaborn as sns

# -----------------------------------------KERAS Imports ------------------------------------
from keras import Input, metrics
from keras.layers import Dense,Dropout
from keras.models import Sequential, Model
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
# ------------------------------------End of Keras Imports-----------------------------------
import torch
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor
# !pip install pytorch-tabnet
# from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.model_selection import train_test_split
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
from sklearn.metrics import log_loss

In [ ]:
import random
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(88)

# Read Data

In [ ]:
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_nonscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
train_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")

test_features = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
submission = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

## Preprocessing

In [ ]:
train_features['cp_type'].value_counts()

In [ ]:
remove_vehicle = True
# Since if cp_type!=trt_cp, all the targets are 0

if remove_vehicle:
    kept_index = train_features['cp_type']=='trt_cp'
    train = train_features.loc[kept_index].reset_index(drop=True)
    train_targets_scored = train_scored.loc[kept_index].reset_index(drop=True)

train_features["cp_type"] = (train_features["cp_type"]=="trt_cp") + 0
train_features["cp_dose"] = (train_features["cp_dose"]=="D1") + 0

test_features["cp_type"] = (test_features["cp_type"]=="trt_cp") + 0
test_features["cp_dose"] = (test_features["cp_dose"]=="D1") + 0

X_test = test_features.values

In [ ]:
kept_index = train_features['cp_type']==1
train_features = train_features.loc[kept_index].reset_index(drop=True)

kept_index = test_features['cp_type']==1
test_features = test_features.loc[kept_index].reset_index(drop=True)

In [ ]:
train_features.shape, test_features.shape

# Stratified kfold

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit

In [ ]:
train_targets_scored.shape

In [ ]:
train_targets_scored.head()

In [ ]:
# Taken from Chris Deotte folds

SEED = 42

NB_SPLITS = 5  #5
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
targets = train_targets_scored.columns[1:]
# Taken from Chris : https://www.kaggle.com/c/lish-moa/discussion/195195
scored = train_targets_scored.merge(drug, on='sig_id', how='left') 
# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, shuffle=True, 
          random_state=SEED)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, shuffle=True, 
          random_state=SEED)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
scored['fold'] = scored.drug_id.map(dct1)
scored.loc[scored.fold.isna(),'fold'] =\
    scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
scored.fold = scored.fold.astype('int8')

In [ ]:
scored.head()

In [ ]:
scored.shape

## Losses

In [ ]:
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score, log_loss

class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)


## Rank Gauss

In [ ]:
import os
os.chdir("../input/rank-gauss")

from gauss_rank_scaler import GaussRankScaler

In [ ]:
# GENES = [col for col in train_features.columns if col.startswith("g-")]
# CELLS = [col for col in train_features.columns if col.startswith("c-")]

In [ ]:
print(train_features.shape)
print(test_features.shape)

In [ ]:
21948+3624

In [ ]:
data_all = pd.concat([train_features, test_features], ignore_index = True)
cols_numeric = [feat for feat in list(data_all.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]
mask = (data_all[cols_numeric].var() >= 0.7).values
tmp = data_all[cols_numeric].loc[:, mask]
data_all = pd.concat([data_all[["sig_id", "cp_type", "cp_time", "cp_dose"]], tmp], axis = 1)
cols_numeric = [feat for feat in list(data_all.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]

In [ ]:
data_all.shape

In [ ]:
scaler = GaussRankScaler()
data_all[cols_numeric] = scaler.fit_transform(data_all[cols_numeric])

## PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
ncompo_genes = 80
ncompo_cells = 10

In [ ]:
GENES = [col for col in data_all.columns if col.startswith("g-")]
CELLS = [col for col in data_all.columns if col.startswith("c-")]

In [ ]:
pca_genes = PCA(n_components = ncompo_genes,random_state = 42).fit_transform(data_all[GENES])
pca_cells = PCA(n_components = ncompo_cells,random_state = 42).fit_transform(data_all[CELLS])
    
pca_genes = pd.DataFrame(pca_genes, columns = [f"pca_g-{i}" for i in range(ncompo_genes)])
pca_cells = pd.DataFrame(pca_cells, columns = [f"pca_c-{i}" for i in range(ncompo_cells)])
data_all = pd.concat([data_all, pca_genes, pca_cells], axis = 1)

In [ ]:
data_all.shape

### Feature Engineering - Stats

In [ ]:
import tqdm
GENES = [col for col in data_all.columns if col.startswith("g-")]
CELLS = [col for col in data_all.columns if col.startswith("c-")]

for stats in tqdm.tqdm(["sum", "mean", "std", "kurt", "skew"]):
    data_all["g_" + stats] = getattr(data_all[GENES], stats)(axis = 1)
    data_all["c_" + stats] = getattr(data_all[CELLS], stats)(axis = 1)    
    data_all["gc_" + stats] = getattr(data_all[GENES + CELLS], stats)(axis = 1)

In [ ]:
data_all.head()

In [ ]:
# train_df and test_df
data_all_copy = data_all.copy()
features_to_drop = ["sig_id"]
data_all.drop(features_to_drop, axis = 1, inplace = True)

train_df = data_all[: train_features.shape[0]]
train_df.reset_index(drop = True, inplace = True)
# The following line it's a bad practice in my opinion, targets on train set
#train_df = pd.concat([train_df, targets], axis = 1)
test_df = data_all[train_df.shape[0]: ]
test_df.reset_index(drop = True, inplace = True)

In [ ]:
# Having a backup in the end
train_df_copy = data_all_copy[: train_features.shape[0]]
train_df_copy.reset_index(drop = True, inplace = True)
# The following line it's a bad practice in my opinion, targets on train set
#train_df = pd.concat([train_df, targets], axis = 1)
test_df_copy = data_all_copy[train_df.shape[0]: ]
test_df_copy.reset_index(drop = True, inplace = True)

In [ ]:
test_df_copy.shape

In [ ]:
train_df.shape

In [ ]:
test_df.shape

## Pretrainer

In [ ]:
train_features = train_df.copy()
test_features = test_df.copy()

In [ ]:
print(train_features.shape)
train_features.head()

In [ ]:
# train_features.drop(columns=["sig_id"], inplace=True)
# test_features.drop(columns=['sig_id'], inplace=True)

In [ ]:
test_features.head()

In [ ]:
train_features.values[:,1:]

In [ ]:
from pytorch_tabnet.pretraining import TabNetPretrainer

BS=1024
MAX_EPOCH=101

N_D = 128
N_A = 48  #32
N_INDEP = 1
N_SHARED = 1
N_STEPS = 3

tabnet_params = dict(n_d=N_D, n_a=N_A, n_steps=N_STEPS,  #0.2,
                         n_independent=N_INDEP, n_shared=N_SHARED,
                         lambda_sparse=0., optimizer_fn=torch.optim.Adam,
                         optimizer_params=dict(lr=2e-2),
                         mask_type="entmax",
                         scheduler_params=dict(mode="min",
                                               patience=5,
                                               min_lr=1e-5,
                                               factor=0.9,),
                         scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,                         
                         verbose=10,
                         )

pretrainer = TabNetPretrainer(**tabnet_params)
#test_features.values[:,1:]  on 25th Nov
pretrainer.fit(X_train=test_features.values[:,1:], #  np.vstack([train.values[:,1:], test.values[:,1:]])
          eval_set=[train_features.values[:,1:]],
          max_epochs=MAX_EPOCH,
          patience=20, batch_size=BS, virtual_batch_size=128, #128,
          num_workers=0, drop_last=True,
          pretraining_ratio=0.8)

## Cross Validation starting from pretrained weights

In [ ]:
test_features.head()

In [ ]:
#X_test = test_features.values[:,1:]
X_test = test_features.values[:,1:]

In [ ]:
import tensorflow as tf

In [ ]:
train_targets_scored.head()

In [ ]:
scored.shape

In [ ]:
train_features.head()

In [ ]:
scores_auc_all= []
test_cv_preds = []
oof_preds = []
oof_targets = []
scores = []
scores_auc = []
NB_FOLD = 5
for fold_nb in range(NB_FOLD):
    train_idx = scored[scored.fold!=fold_nb].index
    val_idx = scored[scored.fold==fold_nb].index

    print("FOLDS : ", fold_nb)
    if fold_nb >= NB_FOLD:
        break
    ## model
#     X_train, y_train = train_features.values[train_idx, 1:], train_scored.values[train_idx, 1:].astype(float) #[:,simple_tasks].astype(float)
#     X_val, y_val = train_features.values[val_idx, 1:], train_scored.values[val_idx, 1:].astype(float) # [:,simple_tasks].astype(float)
    
    X_train, y_train = train_features.values[train_idx, 1:], train_targets_scored.values[train_idx, 1:].astype(float) #[:,simple_tasks].astype(float)
    X_val, y_val = train_features.values[val_idx, 1:], train_targets_scored.values[val_idx, 1:].astype(float) # [:,simple_tasks].astype(float)
    MAX_EPOCH=200
    BS=1024

    tabnet_params = dict(n_d=N_D, n_a=N_A, n_steps=N_STEPS,
                         n_independent=N_INDEP, n_shared=N_SHARED,
                         gamma=1.0,
                         lambda_sparse=0., optimizer_fn=torch.optim.Adam, # 
                         optimizer_params=dict(lr=2e-2, # 2e-2
                                               weight_decay=1e-5
                                              ),
                         mask_type="entmax",
                         scheduler_params=dict(max_lr=0.05,
                                              steps_per_epoch=int(X_train.shape[0] / BS),
                                              epochs=MAX_EPOCH,
                                              is_batch_level=True),
                         scheduler_fn=torch.optim.lr_scheduler.OneCycleLR,
                         verbose=10,
                         )

    model = TabNetRegressor(**tabnet_params)

    model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=BS, virtual_batch_size=128, #128,
              num_workers=1, drop_last=True,
              from_unsupervised=pretrainer,            
              loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
        ## save oof to compute the CV later 
    preds_val = model.predict(X_val)
    # Apply sigmoid to the predictions
    preds =  1 / (1 + np.exp(-preds_val))
    score = np.min(model.history["val_logits_ll"])
    scores.append(score)
    oof_preds.append(preds)
    oof_targets.append(y_val)

#     name = cfg.save_name + f"_fold{fold_nb}"
#     model.save_model(name)    

    # preds on test
    preds_test = model.predict(X_test)
#     preds_test = model.predict(X_test[:,1:])
    test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)

In [ ]:
train_features.shape

## Consolidating results

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
aucs = []
for task_id in range(oof_preds_all.shape[1]):
    aucs.append(roc_auc_score(y_true=oof_targets_all[:, task_id],
                              y_score=oof_preds_all[:, task_id]))
print(f"Overall AUC : {np.mean(aucs)}")
print(f"Average CV : {np.mean(scores)}")

In [ ]:
!pwd

In [ ]:
test_features.head()

In [ ]:
test_df_copy.head()

In [ ]:
all_feat = [col for col in submission.columns if col not in ["sig_id"]]
test_df_copy[all_feat] = test_preds_all.mean(axis=0)
column_names = []
column_names.append("sig_id")
column_names.extend(all_feat)


In [ ]:
submission_df = pd.merge(submission['sig_id'] ,test_df_copy[column_names], on="sig_id",how="left")
submission_df.fillna(0,inplace=True)
submission_df.head()

In [ ]:
os.chdir("..")

In [ ]:
!pwd

In [ ]:
submission_df.to_csv('../working/submission.csv', index=None)

## Decompose features using AutoEncoders Keras

In [ ]:
# combine_df_rescaled.head()

### Defining AE

In [ ]:
# combine_df_rescaled.drop(['sig_id','cp_time','cp_dose','kfold','is_train','index'],axis=1).shape

In [ ]:
# combine_df_rescaled.drop(['sig_id','cp_time','cp_dose','kfold','is_train','index'],axis=1).head()

In [ ]:
# METRICS = [metrics.RootMeanSquaredError(name='rms'),metrics.MeanAbsoluteError(name='mae')]

# encoded_dim = 60   # 60 before

# input_df = Input(shape=(772,))
# encode_layer1 = Dense(encoded_dim * 4, activation='swish')(input_df)
# encode_layer2 = Dense(encoded_dim * 3, activation='swish')(encode_layer1)
# encode_layer3 = Dense(encoded_dim * 2, activation='swish')(encode_layer2)
# bottleneck = Dense(encoded_dim, activation='swish')(encode_layer3)   # this is the main layer for encoding dim 60

# decode_layer1 = Dense(encoded_dim * 2, activation='swish')(bottleneck)
# decode_layer2 = Dense(encoded_dim * 3, activation='swish')(decode_layer1)
# decode_layer3 = Dense(encoded_dim * 4, activation='swish')(decode_layer2)
# output = Dense(772, activation='swish')(decode_layer3)

# autoencoders = Model(input_df, output)  # To check the  mse of the model
# encoders = Model(input_df, bottleneck)  # To reduce the dimensions

# autoencoders.compile(optimizer='adam', loss='mean_squared_error',metrics=[METRICS])

In [ ]:
# g_features = [col for col in list(combine_df_rescaled.columns) if 'g-' in col]
# g_features_combined = combine_df_rescaled[g_features]

In [ ]:
# g_features_combined.shape

In [ ]:
# X_train = g_features_combined.iloc[:22237]
# X_valid = g_features_combined.iloc[22237:]

# reduce_lr = ReduceLROnPlateau(monitor='val_logloss', factor=0.1, verbose=0,mode='min',
#                               patience=3, min_lr=1E-7)
# early_st = EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=7, verbose=0, mode='min',
#     baseline=None, restore_best_weights=True)

# autoencoders.fit(X_train, X_train,
#                 epochs=100,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(X_valid, X_valid),
#                 callbacks =[reduce_lr, early_st]
#                 )

# X_reduced = encoders.predict(g_features_combined)

# col = []
# for i in range(60):
#     col_name = f"gae_{i}"
#     col.append(col_name)
# X_reduced_df = pd.DataFrame(X_reduced, columns = col)

# #non_reduced = combine_df_rescaled[['sig_id','cp_type','cp_time','cp_dose','kfold','is_train','index']]
# combine_df_prescaled_pca = pd.concat([combine_df_rescaled, X_reduced_df],axis=1)  # autoencoders

In [ ]:
# METRICS = [metrics.RootMeanSquaredError(name='rms'),metrics.MeanAbsoluteError(name='mae')]

# encoded_dim = 15   # 15 before

# input_df = Input(shape=(100,))
# encode_layer1 = Dense(encoded_dim * 4, activation='swish')(input_df)
# encode_layer2 = Dense(encoded_dim * 3, activation='swish')(encode_layer1)
# encode_layer3 = Dense(encoded_dim * 2, activation='swish')(encode_layer2)
# bottleneck = Dense(encoded_dim, activation='swish')(encode_layer3)   # this is the main layer for encoding dim 60

# decode_layer1 = Dense(encoded_dim * 2, activation='swish')(bottleneck)
# decode_layer2 = Dense(encoded_dim * 3, activation='swish')(decode_layer1)
# decode_layer3 = Dense(encoded_dim * 4, activation='swish')(decode_layer2)
# output = Dense(100, activation='swish')(decode_layer3)

# autoencoders = Model(input_df, output)  # To check the  mse of the model
# encoders = Model(input_df, bottleneck)  # To reduce the dimensions

# autoencoders.compile(optimizer='adam', loss='mean_squared_error',metrics=[METRICS])

# c_features = [col for col in list(combine_df_prescaled_pca.columns) if 'c-' in col]
# c_features_combined = combine_df_prescaled_pca[c_features]


# X_train = c_features_combined.iloc[:22237]
# X_valid = c_features_combined.iloc[22237:]

# reduce_lr = ReduceLROnPlateau(monitor='val_logloss', factor=0.1, verbose=0,mode='min',
#                               patience=3, min_lr=1E-7)
# early_st = EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=7, verbose=0, mode='min',
#     baseline=None, restore_best_weights=True)

# autoencoders.fit(X_train, X_train,
#                 epochs=100,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(X_valid, X_valid),
#                 callbacks =[reduce_lr, early_st]
#                 )

# X_reduced = encoders.predict(c_features_combined)

# col = []
# for i in range(15):
#     col_name = f"cae_{i}"
#     col.append(col_name)
# X_reduced_df = pd.DataFrame(X_reduced, columns = col)

# #non_reduced = combine_df_rescaled[['sig_id','cp_type','cp_time','cp_dose','kfold','is_train','index']]
# combine_df_prescaled_pca = pd.concat([combine_df_prescaled_pca, X_reduced_df],axis=1)  # autoencoders

In [ ]:
# combine_df_prescaled_pca.shape

## Decompose features using PCA as there are lots of features
n_components we will try to setup using variances 0.95 or 0.99

### PCA for g- features

In [ ]:
# g_features = [col for col in list(combine_df_rescaled.columns) if 'g-' in col]
# g_features_combined = combine_df_rescaled[g_features]

In [ ]:
# ## using 30 components as per discussion forum
# pca = PCA(n_components = 30)
# pca.fit(g_features_combined.values)
# reduced_g_features = pca.fit_transform(g_features_combined.values)

In [ ]:
# pca = PCA(n_components = 0.99)
# pca.fit(g_features_combined.values)
# reduced_g_features = pca.fit_transform(g_features_combined.values)

In [ ]:
# print(g_features_combined.shape)
# print(reduced_g_features.shape)
# ## looks like more columns again

In [ ]:
# pca = PCA(n_components = 0.95)
# pca.fit(g_features_combined.values)
# reduced_g_features = pca.fit_transform(g_features_combined.values)
# print(g_features_combined.shape)
# print(reduced_g_features.shape)

We will go with 0.95 variance as this has significantly reduced 270 columns approximately

In [ ]:
# # reduced_g_features to columns in pandas dataframe
# g_columns_components = []
# for i in range(reduced_g_features.shape[1]):
#     g_columns_components.append("g-pca-{}".format(i))
# g_component_df = pd.DataFrame(reduced_g_features,columns=g_columns_components)

In [ ]:
# combine_df_prescaled_pca = pd.concat([combine_df_rescaled, g_component_df],axis=1)

In [ ]:
# combine_df_rescaled.shape

In [ ]:
# combine_df_prescaled_pca.shape

### PCA for c- features

In [ ]:
# c_features = [col for col in list(combine_df_rescaled.columns) if 'c-' in col]
# c_features_combined = combine_df_rescaled[c_features]

In [ ]:
# # using 4 components as per discussion forum
# pca = PCA(n_components = 4)
# pca.fit(c_features_combined.values)
# reduced_c_features = pca.fit_transform(c_features_combined.values)
# print(c_features_combined.shape)
# print(reduced_c_features.shape)

In [ ]:
# pca = PCA(n_components = 0.99)
# pca.fit(c_features_combined.values)
# reduced_c_features = pca.fit_transform(c_features_combined.values)
# print(c_features_combined.shape)
# print(reduced_c_features.shape)

In [ ]:
# pca = PCA(n_components = 0.95)
# pca.fit(c_features_combined.values)
# reduced_c_features = pca.fit_transform(c_features_combined.values)
# print(c_features_combined.shape)
# print(reduced_c_features.shape)

Again 95% variance is chosen as it has significantly decomposed many columns

In [ ]:
# # reduced_c_features to columns in pandas dataframe
# c_columns_components = []
# for i in range(reduced_c_features.shape[1]):
#     c_columns_components.append("c-pca-{}".format(i))
# c_component_df = pd.DataFrame(reduced_c_features,columns=c_columns_components)

In [ ]:
# combine_df_prescaled_pca = pd.concat([combine_df_prescaled_pca, c_component_df],axis=1)

## Combining all together into one dataframe

In [ ]:
# combine_df_rescaled_pca = pd.concat([combine_df_rescaled[['sig_id','cp_type','cp_time','cp_dose','kfold','is_train','index']],g_component_df],axis=1)
# combine_df_rescaled_pca = pd.concat([combine_df_rescaled_pca,c_component_df],axis=1)

In [ ]:
# combine_df_rescaled.shape

In [ ]:
# combine_df_rescaled_pca.shape

In [ ]:
# combine_df_rescaled_pca.head()

# Feature Selection

In [ ]:
# combine_df_prescaled_pca_copy = combine_df_prescaled_pca.copy()

In [ ]:
# combine_df_prescaled_pca = combine_df_prescaled_pca_copy.copy()

In [ ]:
# combine_df_prescaled_pca.shape

In [ ]:
# var_t = VarianceThreshold(0.5)
# feature_selected_g_c = var_t.fit_transform(combine_df_prescaled_pca.drop(['sig_id','cp_time','cp_dose','kfold','is_train','index'],axis=1))
# cols = []
# print(feature_selected_g_c.shape)
# for i in range(feature_selected_g_c.shape[1]):
#     cols.append("g_c_{}".format(i))
# feature_selected_var_thresh = pd.DataFrame(feature_selected_g_c,columns=cols)
# combine_df_prescaled_pca = pd.concat([combine_df_prescaled_pca[['sig_id','cp_time','cp_dose','kfold','is_train','index']], feature_selected_var_thresh],axis=1)

# Model

In [ ]:
# combine_df_prescaled_pca.shape

## We will predict nonscored features

In [ ]:
# import os
# os.chdir("../../input/tabnet-v1/tabnet-develop")

In [ ]:
# from pytorch_tabnet.metrics import Metric

In [ ]:
# from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
# combine_df_prescaled_pca.head()

In [ ]:
# train = combine_df_prescaled_pca[combine_df_prescaled_pca['is_train'] == 1]
# test = combine_df_prescaled_pca[combine_df_prescaled_pca['is_train'] == 0]

# ### Defining the categorical columns and dims

# categorical_columns = ['cp_time','cp_dose']
# categorical_dims = {'cp_time':3,'cp_dose':2}

# unused_feats = ['sig_id','index','kfold','is_train']

# features = [ col for col in train.columns if col not in unused_feats] 

# cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

# cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]


In [ ]:
# cat_dims

## Model Params

In [ ]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
# BS=256
# MAX_EPOCH=101

# N_D = 128
# N_A = 32
# N_INDEP = 1
# N_SHARED = 1
# N_STEPS = 3

# tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.3,
#                      lambda_sparse=0, optimizer_fn=torch.optim.Adam,
#                      optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
#                      mask_type='entmax',
#                      scheduler_params=dict(mode="min",
#                                            patience=5,
#                                            min_lr=1e-5,
#                                            factor=0.9,),
#                    #  scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
#                      verbose=10,
#                      )

# pretrainer = TabNetPretrainer(**tabnet_params)

# pretrainer.fit(X_train=test.drop(['sig_id','kfold','is_train','index'], axis=1).values, #  np.vstack([train.values[:,1:], test.values[:,1:]])
#           eval_set=[train.drop(['sig_id','kfold','is_train','index'], axis=1).values],
#           max_epochs=MAX_EPOCH,
#           patience=20, batch_size=BS, virtual_batch_size=32, #128,
#           num_workers=0, drop_last=True,
#           pretraining_ratio=0.8)

In [ ]:
# clf = TabNetRegressor(**tabnet_params)

In [ ]:
# clf = TabNetRegressor(cat_dims=cat_dims, cat_emb_dim=[2,3,2], cat_idxs=cat_idxs)

## Model

In [ ]:
# reduce_lr = ReduceLROnPlateau(monitor='val_logloss', factor=0.1, verbose=0,mode='min',
#                               patience=3, min_lr=1E-7)
# early_st = EarlyStopping(monitor='val_logloss', min_delta=1E-4, patience=7, verbose=0, mode='min',
#     baseline=None, restore_best_weights=True)

In [ ]:
# from sklearn.metrics import log_loss
# from pytorch_tabnet.metrics import Metric
# from sklearn.metrics import roc_auc_score, log_loss

# class LogitsLogLoss(Metric):
#     """
#     LogLoss with sigmoid applied
#     """

#     def __init__(self):
#         self._name = "logits_ll"
#         self._maximize = False

#     def __call__(self, y_true, y_pred):
#         """
#         Compute LogLoss of predictions.

#         Parameters
#         ----------
#         y_true: np.ndarray
#             Target matrix or vector
#         y_score: np.ndarray
#             Score matrix or vector

#         Returns
#         -------
#             float
#             LogLoss of predictions vs targets.
#         """
#         logits = 1 / (1 + np.exp(-y_pred))
#         aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
#         return np.mean(-aux)

In [ ]:
# def create_df(preds):
#     col_names = []
#     for i in range(206):
#         col_names.append(str(i))
#     df = pd.DataFrame(preds.reshape(-1,206), columns=col_names)
#     return df
# def perform_fitting_fold_tabnet(train, test, train_scored, fold, NB_TASKS = 206):
#     train_v1 = train[train['kfold'] == fold]
#     train_scored_v1 = train_scored[train_scored['sig_id'].isin(train_v1['sig_id'].unique())]
    
#     valid = train[train['kfold'] != fold]
#     valid_scored = train_scored[train_scored['sig_id'].isin(valid['sig_id'].unique())]
    
#     print(valid)
#     X_train = train_v1.drop(['sig_id','index','kfold','is_train'],axis=1)
#     y_train = train_scored_v1.drop("sig_id",axis=1)
    
#     X_valid = valid.drop(['sig_id','index','kfold','is_train'],axis=1)
#     y_valid = valid_scored.drop("sig_id",axis=1)
    
    
#     X_test = test.drop(['sig_id','index','kfold','is_train'],axis=1)
    
#     clf.fit(X_train.values,y_train.values,
#             eval_set=[(X_valid.values, y_valid.values)],
#             eval_name = ["val"],
#             eval_metric = ["logits_ll"],
#             max_epochs=100, patience=20, batch_size=1024, virtual_batch_size=128,
#             loss_fn=torch.nn.functional.binary_cross_entropy_with_logits,
#             from_unsupervised=pretrainer)
    
#     preds = clf.predict(X_test.values)
    
#     # perform sigmoid
#     preds_sig =  1 / (1 + np.exp(-preds))
    
#     df = create_df(preds_sig)
#     return df

## Model2

In [ ]:
# train.shape

In [ ]:
# p_min = 0.0005
# p_max = 0.9995
# import tensorflow as tf
# from keras import backend 
# def logloss(y_true, y_pred):
#     y_pred = tf.clip_by_value(y_pred,p_min,p_max)
#     return -backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))

In [ ]:
# from keras import losses

In [ ]:
# def create_model(input_shape):
#     input_df = Input(shape=(input_shape,))
#     layer1 = Dense(512, activation="elu")(input_df)
#     layer2 = Dense(1024, activation="elu")(layer1)
#     layer3 = Dense(512, activation="elu")(layer2)
#     layer4 = Dense(2048, activation="elu")(layer3)
#     output = Dense(206, activation="sigmoid")(layer4)

#     model2 = Model(input_df, output) 
#     model2.compile(optimizer='adam', loss=losses.BinaryCrossentropy(label_smoothing=0.0015),metrics=logloss)
#     return model2

In [ ]:
# def perform_fitting_fold_model2(train, test, train_scored, fold, NB_TASKS = 206):
#     train_v1 = train[train['kfold'] == fold]
#     train_scored_v1 = train_scored[train_scored['sig_id'].isin(train_v1['sig_id'].unique())]
    
#     valid = train[train['kfold'] != fold]
#     valid_scored = train_scored[train_scored['sig_id'].isin(valid['sig_id'].unique())]

#     X_train = train_v1.drop(['sig_id','index','kfold','is_train'],axis=1)
#     y_train = train_scored_v1.drop("sig_id",axis=1)
    
#     X_valid = valid.drop(['sig_id','index','kfold','is_train'],axis=1)
#     y_valid = valid_scored.drop("sig_id",axis=1)
    
#     X_test = test.drop(['sig_id','index','kfold','is_train'],axis=1)
#     input_shape = X_train.shape[1]
#     model2 = create_model(input_shape)
#     model2.fit(X_train.values,y_train.values,epochs=200,batch_size=256,
#                validation_data = (X_valid.values,y_valid.values),
#                callbacks =[reduce_lr, early_st]
#               )
    
#     preds = model2.predict(X_test.values)
    
#     df = create_df(preds)
#     return df

In [ ]:
# train.shape

In [ ]:
# n_seeds = 6
# np.random.seed(1)
# seeds = np.random.randint(0,100,size=n_seeds)

# preds_df_list = []
# for i,seed in enumerate(seeds):
#     for j in range(5):
#         if(seed%2 == 0):
# #             preds = perform_fitting_fold_model2(train,test,train_scored,j)
# #             preds_df_list.append(preds)
#             preds = perform_fitting_fold_tabnet(train,test,train_scored,j)
#             preds_df_list.append(preds)
#         else:
#             preds = perform_fitting_fold_tabnet(train,test,train_scored,j)
#             preds_df_list.append(preds)
# #             preds = perform_fitting_fold_model2(train,test,train_scored,j)
# #             preds_df_list.append(preds)

In [ ]:
# len(preds_df_list)

In [ ]:
# df_list = []
# df= pd.DataFrame()
# for i in range(0, len(preds_df_list),5):
#     for j in range(preds_df_list[0].shape[1]):
#         cols_name = str(j)
#         df[cols_name] = (preds_df_list[i].iloc[:,j] + preds_df_list[i+1].iloc[:,j] + preds_df_list[i+2].iloc[:,j] + preds_df_list[i+3].iloc[:,j] + preds_df_list[i+4].iloc[:,j]) / 5
#     df_list.append(df)
#     df = pd.DataFrame()

In [ ]:
# ############### For seeds###############
# df_sub = pd.DataFrame()
# for j in range(df_list[0].shape[1]):
#     cols_name = str(j)
#     df_sub[cols_name] = (df_list[0].iloc[:,j] + df_list[1].iloc[:,j] + df_list[2].iloc[:,j] + df_list[3].iloc[:,j] + df_list[4].iloc[:,j] + df_list[5].iloc[:,j]) / 6


In [ ]:
# df_sub.columns = submission.columns[1:]

In [ ]:
# test.reset_index(inplace=True)

In [ ]:
# df_sub = pd.concat([test['sig_id'], df_sub],axis=1)

In [ ]:
# df_sub.shape

In [ ]:
# submission.shape

In [ ]:
# mysub = pd.merge(submission['sig_id'], df_sub, on="sig_id",how='left')

In [ ]:
# mysub.fillna(0, inplace=True)

In [ ]:
# mysub = mysub[submission.columns]

In [ ]:
# mysub.shape

In [ ]:
# os.chdir("/kaggle/working")

In [ ]:

# mysub.to_csv("submission.csv",index=False)